# `II. Field Simulation` 
This .pynb file reads `./inter_results/mesh_result.pkl` generated by `./I_Mesh_Processing.ipynb`, and simulates electric field distribution. The result is stored in `./inter_results/field_result.pkl`.

In this file we compute electric field or potential on some spatial points(called `grid`) for all electrode voltage configurations(i.e. one eletrode is 1V and the rest 0V).Major calculations calls `fastlap` C library which uses a pre-conditioned, adaptive, multipole-accelerated algorithm for solving Laplace problem. Two parameters control multipole acceleration.
+ num_mom, the number of multipole
+ num_lev, the number of levels in the hierarchical spatial decomposition.  
num_lev=1 means direct computation without multipole acceleration. See fastlap ug.pdf and README.rst.


In `run_job` function, `job` is `Configuration` instance and `grid` is discretirized spatial grid (not the mesh). The general workflow (also the routine of BEM method) are:  
1. `solve_singularities()` solves charge distributions by iterative methods to make it consistent with one electrode at 1V and others at 0V (unit potentials). `adapt_mesh()` refines meshes adaptively to achieve certain precision while solving sigulartities.
2. Compute potentials on given grid points by `simulate()`, based on the charge distributions gotten previously.
3. Potential data of each unit potential are saved seperately to a `Result` instance, and also export to VTK files.
4. Return total accumulated charge per electrode in the end.

notes:
* here we invoke `multiprocessing.set_start_method("fork")` for compatibility with python version 3.9
* This is the most time consuming part in our workflow. Running following codes in a `.py` file instead of `.ipynb` may be helpful. Closing other apps in your laptop is helpful. Using HPC is also an option. 
* for reference, this file in my laptop(macbook pro, 2 physical kernels) runs 800s
* This `.ipynb` file will generate a `./.vtk` file for intermediate files, which can be simply ignored.

## (1) define grid
First of all, we define some spatial points for observing the field, called grid



In [1]:
from time import time
import pickle
import numpy as np
import os
import sys
import vtk
sys.path.append('/home/sqip/Documents/github/bem')
from bem import Electrodes, Sphere, Mesh, Grid, Configuration, Result
import multiprocessing 
# multiprocessing.set_start_method("fork")
from utils.helper_functions import *

radius= 500e-3
area = 1e-5
file = 'mit_LL'
file_in_name = 'inter_results/inter_results_mit/'+file+'_'+str(radius)+'_'+str(area)+'.pkl'
vtk_out = "inter_results/inter_results_mit/.vtks/"+file
file_out_name = 'inter_results/inter_results_mit/'+file+'_'+str(radius)+'_'+str(area)+'_simulation'


#open the mesh that will be used for simulation
with open(file_in_name,'rb') as f:
    mesh_unit,xl,yl,zl,mesh,electrode_names= pickle.load(f) # import results from mesh processing



# grid to evalute potential and fields atCreate a grid in unit of scaled length mesh_unit. Only choose the interested region (trap center) to save time.
s = 1e-3
# L = 102e-6
Lx, Ly, Lz = 15*1e-3,15e-3,15*1e-3# in the unit of scaled length mesh_unit
# xl,yl,zl = -3.75*1e-3,72*1e-3,270*1.0e-3
xl,yl,zl = 0.0e-3,50.0e-3,0.0e-3
sx,sy,sz = s,s,s
print("done")
# ni is number of grid points, si is step size. To  on i direction you need to fix ni*si.
nx, ny, nz = [int(Lx/sx),int(Ly/sy),int(Lz/sz)]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(xl,yl,zl), step=(sx, sy, sz), shape=(nx,ny,nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("lowval",grid.indices_to_coordinates([0,0,0]))
print("Grid center index", grid.indices_to_coordinates([nx/2,ny/2,nz/2]))
print("gridpts:",nx*ny*nz)


done
Size/l: 0.015 0.015 0.015
Step/l: 0.001 0.001 0.001
Shape (grid point numbers): 15 15 15
lowval [-0.007  0.043 -0.007]
Grid center index [0.0005 0.0505 0.0005]
gridpts: 3375


## (2) run jobs

evaluate electric potential for each configurations: one electrode at 1V, the rest 0V. Set `pmap` as `multiprocessing.Pool().map` for parallel computing. For serial map, use `map`.

In [2]:
from helper_functions.helper_functions import *
jobs = list(Configuration.select(mesh,"DC.*","RF"))    # select() picks one electrode each time.
# run the different electrodes on the parallel pool
pmap = multiprocessing.Pool().map # parallel map
#pmap = map # serial map
t0 = time()
# range(len(jobs))
def run_map():
    list(pmap(run_job, ((jobs[i], grid, vtk_out,i,len(jobs)) for i in np.arange(len(jobs)))))
    print("Computing time: %f s"%(time()-t0))
    # run_job casts a word after finishing ea"ch electrode.

run_map()


starting job 1 out of 21
starting job 2 out of 21
starting job 3 out of 21
starting job 4 out of 21
starting job 5 out of 21
starting job 6 out of 21
starting job 7 out of 21
starting job 8 out of 21
starting job 9 out of 21

starting job 10 out of 21starting job 11 out of 21
starting job 12 out of 21
starting job 13 out of 21
starting job 14 out of 21starting job 15 out of 21

starting job 16 out of 21


In [3]:
ele = 'DC5'
data_name = "%s_%s.vtk" % (vtk_out, ele)
import pyvista as pv
data = pv.UniformGrid(data_name)
scalar_name = 'potential'
avals = data[scalar_name]
input_range = [avals.min(),avals.max()]
Result.view(vtk_out, ele,isosurfaces = 100, rng = input_range) # add electrode name between '' for result view

In [3]:
# for x in np.arange(0,4):
data_name = "%s_%s.vtk" % (vtk_out, 'RF')
import pyvista as pv 
data = pv.UniformGrid(data_name)
scalar_name = 'field_square'

avals = data[scalar_name]
input_range = [avals.min(),avals.max()*5]
Result.view(vtk_out, 'RF',isosurfaces=100, rng=  input_range) # add electrode name between '' for result view

In [ ]:
write_pickle(vtk_out,file_out_name,grid,electrode_names)